In [131]:
import os, re
print ("Hello World!")

Hello World!


In [132]:

class Cell:
    def __init__(self, val=-1):
        self.value = val
        self.possible_values = None
        # if (val == -1) :
        #    self.possible_values = [v for v in range (9)] 

        self.prev_r = -1
        self.prev_c = -1

    def __repr__(self):
        # return (str(self.value))
        if (self.possible_values is None or len(self.possible_values) == 0):
            return (str(self.value))
        elif (self.value != -1):
            return (f"{self.value} [{self.possible_values}]")
        else:
            return (f"{self.value} {self.possible_values}")
            # return (f"{self.possible_values}")
    
    def __str__(self):
        if (self.possible_values is None or len(self.possible_values) == 0):
            return (str(self.value))
        elif (self.value != -1):
            return (f"{self.value} [{self.possible_values}]")
        else:
            return (f"{self.value} {self.possible_values}")
            # return (f"{self.possible_values}")

    def get_val(self):
        return (self.value)

    
    
class Grid:
    def __init__(self):
        self.raw_grid = [[Cell() for c in range(9)] for r in range (9)]
        self.grid_t = []
        
        # Stack of cells for which a possible value was attempted.
        # Will contain a list of coordinate [r, c] of the last cell that we attempted to resolve
        self.prev_cell = []

    def populate(self, new_grid):
        self.raw_grid = [ [Cell(new_grid[r][c]) for c in range (9)] for r in range (9) ]

    def transpose(self):
        if (len(self.grid_t) == 0):
            self.grid_t = [ [self.raw_grid[r][c] for r in range(9)] for c in range (9) ]
    
    def is_set_valid(self, cells):
        values = [i.get_val() for i in cells]
        all_values = [i for i in range(1, 10)]
        valid = True if len(set(values).symmetric_difference(all_values)) == 0 else False
        # if valid == False:
        #    print ('{} contains non-unique elements'.format(values))
        return (valid)

    def check_grid(self):
        # Check all rows
        r_idx = 0
        for r in self.raw_grid:
            valid = self.is_set_valid(r)
            if (valid == False):
                # print ('Row Index: {} = {} does not contain all digits from 0-9'.format(r_idx, r))
                return (False)
            r_idx += 1

        # Check all columns
        self.transpose()
        c_idx = 0
        for c in self.grid_t:
            valid = self.is_set_valid(c)
            if (valid == False):
                # print ('Column Index: {} = {} does not contain all digits from 0-9'.format(c, c_idx))
                return (False)
            c_idx += 1


        # Check all sub-grids
        row_idx = 0
        col_idx = 0
        for grid_c in range (3):
            for grid_r in range (3):
                subgrid = [[self.raw_grid[r][c] for c in range (col_idx, col_idx+3)] for r in range (row_idx, row_idx+3)]
                # print (subgrid)
                subgrid_flat = [i for r in subgrid for i in r]
                valid = self.is_set_valid(subgrid_flat)
                if (valid == False):
                    # print ('Subgrid {} x {} = {} does not contain all digits from 0-9'.format(grid_r, grid_c, subgrid))
                    return (False)
                col_idx += 3

            col_idx = 0
            row_idx += 3

        # print ('\nGrid is good!')
        return (True)
        
        
    def print(self):
        for r in self.raw_grid:
            print (r)

    def print_transpose(self):
        if len(self.grid_t) == 0: 
            self.transpose()

        for r in self.grid_t:
            print (r) 

    # Solver functions
    # For a given cell, we need to know what's in the rows, columns corresponding to the subgrid the cell is in, and the subgrid itself.
    # Decision: 
    # - Loop through all empty cells
    # - For an empty cell, 
    def old_solve(self):
        while (self.check_grid() == False):
            for r in range (9):
                for c in range (9):
                    v = self.raw_grid[r][c] # Pass by reference
                    if v.get_val() != -1:
                        continue
                    row = self.raw_grid[r]

                    self.transpose()
                    col = self.grid_t[c]

                    row_idx = int(r / 3) * 3 
                    col_idx = int(c / 3) * 3; 
                    subgrid = [[self.raw_grid[r][c] for c in range (col_idx, col_idx+3)] for r in range (row_idx, row_idx+3)]
                    subgrid_flat = [i for r in subgrid for i in r]

                    # print ("Value = {}, Row = {}, Column = {}, subgrid = {}".format(v, row, col, subgrid))

                    # TODO FIXME all_poss = [i for i in range (10)] # For this problem it is in 0-9
                    all_poss = [i for i in range (1, 10)] # For an actual sudoku, this would be 1-9
                    # print (all_poss)
                    cannot_be_values = row + col + subgrid_flat
                    unique_cannot_be_values = []
                    for ii in cannot_be_values:
                        if ii.get_val() not in unique_cannot_be_values:
                            unique_cannot_be_values.append(ii.get_val())

                    v.possible_values = list(set(all_poss).symmetric_difference(unique_cannot_be_values))
                    if (-1 in v.possible_values):
                        v.possible_values.remove(-1)
                    
                    # print (v.possible_values)

                    if (len(v.possible_values) == 1):
                        v.value = v.possible_values.pop()



    def generate_possible_values(self, r, c):
        row = self.raw_grid[r]

        self.transpose()
        col = self.grid_t[c]

        row_idx = int(r / 3) * 3 
        col_idx = int(c / 3) * 3; 
        subgrid = [[self.raw_grid[r][c] for c in range (col_idx, col_idx+3)] for r in range (row_idx, row_idx+3)]
        subgrid_flat = [i for r in subgrid for i in r]

        # print ("Value = {}, Row = {}, Column = {}, subgrid = {}".format(v, row, col, subgrid))

        # TODO FIXME all_poss = [i for i in range (10)] # For this problem it is in 0-9
        all_poss = [i for i in range (1, 10)] # For an actual sudoku, this is 1-9
        # print (all_poss)
        cannot_be_values = row + col + subgrid_flat
        unique_cannot_be_values = []
        for ii in cannot_be_values:
            if ii.get_val() not in unique_cannot_be_values:
                unique_cannot_be_values.append(ii.get_val())

        self.raw_grid[r][c].possible_values = list(set(all_poss).symmetric_difference(unique_cannot_be_values))
        if (-1 in self.raw_grid[r][c].possible_values):
            self.raw_grid[r][c].possible_values.remove(-1)
        
        return (self.raw_grid[r][c].possible_values)






In [133]:
def gcd (a, b):
    if b == 0:
        return a
    else:
        return (gcd(b, a % b))

def row_to_n(row):
    row_str = [(lambda x : str(x))(x) for x in row]
    n = ''.join(row_str)
    print (int(n))
    return (int(n))

def list_gcd(arr):
    result = arr[0]
    for n in range (1, len(arr)):
        result = gcd(result, arr[n])
    print (result)
    return (result)


In [134]:
import sys
def solve(grid, get_gcd=False):

    if (grid.check_grid()):
         print ('Grid is good!')

         for r in range (9):
             for c in range (9):
                 grid.raw_grid[r][c].possible_values = None

         grid.print()
         
         if (get_gcd):
             rows_n = []
             for r in grid.raw_grid:
                 num = [cell.get_val() for cell in r]
                 rows_n.append(row_to_n(num))
            
             print ("GCD: ", list_gcd(rows_n))

         return (True)
    
    for r in range (9):
        for c in range (9):
            # For each cell, see which value works; if we reach a point where nothing works, return out of the loop and try again
            cell = grid.raw_grid[r][c]
            if (cell.get_val() == -1):
                poss_values =  grid.generate_possible_values(r, c)
                if len(poss_values) != 0:
                    status = False
                    for v in poss_values:
                        # print (f"Attempting value {v} for cell {r} x {c}")
                        grid.raw_grid[r][c].value = v
                        status = solve(grid, get_gcd)
                        # if (grid.check_grid() == True):
                        if (status == True):
                            return
                    
                    if (grid.check_grid() == False):
                        # print (f"Resetting {r} x {c}")
                        grid.raw_grid[r][c].value = -1
                        return
                        
                else:
                     # print (f"COuld not find any options for cell {r} x {c}. Returning")
                     return


In [135]:
easy_grid = Grid()
easy = [
    [2, 9, -1,   -1, -1, -1,   8, 7, 1],
    [7, 4, 3,   -1, -1, -1,   -1, 5, -1],
    [5, 8, -1,   -1, -1, -1,   -1, 4, 9],

    [-1, -1, 5,   4, 7, -1,   1, 3, -1],
    [-1, 7, -1,   -1, -1, 2,   -1, -1, -1],
    [-1, 2, 9,    -1, -1, -1,   -1, -1, 4],

    [6, -1, -1,   9, -1, 3,   4, -1, 7],
    [4, 3, 2,    6, 8, -1,    -1, -1, -1],
    [9, -1, -1,   2, 5, -1,   6, -1, 3]
]

easy_grid.populate(easy)
easy_grid.print()

solve(easy_grid, True)


[2, 9, -1, -1, -1, -1, 8, 7, 1]
[7, 4, 3, -1, -1, -1, -1, 5, -1]
[5, 8, -1, -1, -1, -1, -1, 4, 9]
[-1, -1, 5, 4, 7, -1, 1, 3, -1]
[-1, 7, -1, -1, -1, 2, -1, -1, -1]
[-1, 2, 9, -1, -1, -1, -1, -1, 4]
[6, -1, -1, 9, -1, 3, 4, -1, 7]
[4, 3, 2, 6, 8, -1, -1, -1, -1]
[9, -1, -1, 2, 5, -1, 6, -1, 3]
Grid is good!
[2, 9, 6, 3, 4, 5, 8, 7, 1]
[7, 4, 3, 8, 9, 1, 2, 5, 6]
[5, 8, 1, 7, 2, 6, 3, 4, 9]
[8, 6, 5, 4, 7, 9, 1, 3, 2]
[3, 7, 4, 1, 6, 2, 5, 9, 8]
[1, 2, 9, 5, 3, 8, 7, 6, 4]
[6, 5, 8, 9, 1, 3, 4, 2, 7]
[4, 3, 2, 6, 8, 7, 9, 1, 5]
[9, 1, 7, 2, 5, 4, 6, 8, 3]
296345871
743891256
581726349
865479132
374162598
129538764
658913427
432687915
917254683
9
GCD:  9


In [136]:
med = Grid()
med_puzzle = [
    [-1, 5, -1,    -1, 8, -1,    -1, 1, 4],
    [-1, -1, -1,    -1, 5, 2,    6, -1, 3],
    [-1, -1, 9,    -1, -1, 6,    -1, -1, -1],

    [6, 3, -1,     8, -1, 5,     2, 4, 7],
    [5, 9, 4,      1, -1, 7,     8, -1, 6],
    [-1, -1, 2,     3, -1, 4,     -1, -1, -1],

    [1, 6, -1,    -1, -1, 8,    -1, -1, -1],
    [-1, -1, -1,   -1, 4, -1,    1, -1, -1],
    [9, -1, 8,     -1, -1, -1,   -1, 6, 2]
]

med.populate(med_puzzle)
med.print()

solve(med)

[-1, 5, -1, -1, 8, -1, -1, 1, 4]
[-1, -1, -1, -1, 5, 2, 6, -1, 3]
[-1, -1, 9, -1, -1, 6, -1, -1, -1]
[6, 3, -1, 8, -1, 5, 2, 4, 7]
[5, 9, 4, 1, -1, 7, 8, -1, 6]
[-1, -1, 2, 3, -1, 4, -1, -1, -1]
[1, 6, -1, -1, -1, 8, -1, -1, -1]
[-1, -1, -1, -1, 4, -1, 1, -1, -1]
[9, -1, 8, -1, -1, -1, -1, 6, 2]
Grid is good!
[2, 5, 6, 7, 8, 3, 9, 1, 4]
[4, 1, 7, 9, 5, 2, 6, 8, 3]
[3, 8, 9, 4, 1, 6, 7, 2, 5]
[6, 3, 1, 8, 9, 5, 2, 4, 7]
[5, 9, 4, 1, 2, 7, 8, 3, 6]
[8, 7, 2, 3, 6, 4, 5, 9, 1]
[1, 6, 5, 2, 3, 8, 4, 7, 9]
[7, 2, 3, 6, 4, 9, 1, 5, 8]
[9, 4, 8, 5, 7, 1, 3, 6, 2]


In [137]:
med = Grid()
med_puzzle = [
    [6, -1, 5,     7, 4, -1,     -1, -1, -1],
    [7, -1, 4,    6, -1, 2,      -1, 5, 8],
    [2, -1, -1,    -1, -1, -1,    6, 7, -1],

    [-1, 2, -1,    4, -1, 5,      7, 8, -1],
    [-1, -1, -1,   -1, -1, -1,    5, 2, 6],
    [-1, -1, -1,   -1, 7, 6,      -1, 3, -1],

    [5, 6, 8,     -1, 9, -1,      -1, 4, -1],
    [-1, 4, -1,   -1, 6, 7,      -1, 9, 5 ],
    [-1, 9, 7,     5, -1, -1,     -1, 6, -1]
]

med.populate(med_puzzle)
med.print()

solve(med)

[6, -1, 5, 7, 4, -1, -1, -1, -1]
[7, -1, 4, 6, -1, 2, -1, 5, 8]
[2, -1, -1, -1, -1, -1, 6, 7, -1]
[-1, 2, -1, 4, -1, 5, 7, 8, -1]
[-1, -1, -1, -1, -1, -1, 5, 2, 6]
[-1, -1, -1, -1, 7, 6, -1, 3, -1]
[5, 6, 8, -1, 9, -1, -1, 4, -1]
[-1, 4, -1, -1, 6, 7, -1, 9, 5]
[-1, 9, 7, 5, -1, -1, -1, 6, -1]
Grid is good!
[6, 8, 5, 7, 4, 9, 3, 1, 2]
[7, 3, 4, 6, 1, 2, 9, 5, 8]
[2, 1, 9, 3, 5, 8, 6, 7, 4]
[9, 2, 6, 4, 3, 5, 7, 8, 1]
[4, 7, 3, 9, 8, 1, 5, 2, 6]
[8, 5, 1, 2, 7, 6, 4, 3, 9]
[5, 6, 8, 1, 9, 3, 2, 4, 7]
[3, 4, 2, 8, 6, 7, 1, 9, 5]
[1, 9, 7, 5, 2, 4, 8, 6, 3]


In [138]:
hard = Grid()
hard_puzzle = [
    [3, -1, -1,    -1, 5, -1,    7, -1, -1],
    [-1, 5, -1,    -1, 1, -1,    3, -1, 6],
    [-1, 8, 6,    -1, -1, -1,    -1, -1, -1],

    [9, -1, 4,    -1, -1, 5,   2, -1, -1],
    [2, -1, -1,    3, 7, -1,    -1, -1, 5],
    [-1, 3, -1,    1, -1, -1,   4, 6, -1],

    [5, -1, -1,     2, -1, -1,    -1, 7, -1],
    [-1, -1, -1,    -1, -1, -1,   9, -1, 3],
    [1, -1, -1,     7, 4, -1,    -1, -1, 2]
]

hard.populate(hard_puzzle)
solve(hard, True)

Grid is good!
[3, 1, 9, 6, 5, 8, 7, 2, 4]
[4, 5, 2, 9, 1, 7, 3, 8, 6]
[7, 8, 6, 4, 3, 2, 5, 1, 9]
[9, 7, 4, 8, 6, 5, 2, 3, 1]
[2, 6, 1, 3, 7, 4, 8, 9, 5]
[8, 3, 5, 1, 2, 9, 4, 6, 7]
[5, 4, 3, 2, 9, 6, 1, 7, 8]
[6, 2, 7, 5, 8, 1, 9, 4, 3]
[1, 9, 8, 7, 4, 3, 6, 5, 2]
319658724
452917386
786432519
974865231
261374895
835129467
543296178
627581943
198743652
9
GCD:  9


In [139]:
ext = Grid()
puzzle = [
    [-1, -1, 6,     1, -1, -1,     -1, -1, -1],
    [-1, -1, 7,    5, -1, -1,     3, -1, -1],
    [-1, -1, 4,     -1, -1, 8,    6, -1, 7],

    [-1, -1, -1,    -1, -1, -1,    -1, -1, -1],
    [-1, -1, 3,     -1, 4, -1,     2, -1, -1],
    [-1, 5, -1,    8, -1, -1,     7, -1, -1],

    [-1, 3, -1,     -1, -1, 1,    -1, 2, -1],
    [-1, 9, -1,    2, -1, -1,    8, -1, -1],
    [-1, -1, -1,     -1, -1, -1,   -1, 6, 1]
]
ext.populate(puzzle)
solve(ext, True)

Grid is good!
[3, 8, 6, 1, 7, 2, 4, 9, 5]
[9, 1, 7, 5, 6, 4, 3, 8, 2]
[5, 2, 4, 3, 9, 8, 6, 1, 7]
[7, 4, 8, 6, 2, 5, 1, 3, 9]
[1, 6, 3, 9, 4, 7, 2, 5, 8]
[2, 5, 9, 8, 1, 3, 7, 4, 6]
[6, 3, 5, 7, 8, 1, 9, 2, 4]
[4, 9, 1, 2, 5, 6, 8, 7, 3]
[8, 7, 2, 4, 3, 9, 5, 6, 1]
386172495
917564382
524398617
748625139
163947258
259813746
635781924
491256873
872439561
9
GCD:  9


In [140]:
ext = Grid()
puzzle = [
    [-1, -1, -1,     1, -1, -1,     -1, -1, -1],
    [-1, -1, -1,    5, -1, -1,     3, -1, -1],
    [-1, -1, -1,     -1, -1, -1,    6, -1, 7],

    [-1, -1, -1,    -1, -1, -1,    -1, -1, -1],
    [-1, -1, -1,     -1, 4, -1,     2, -1, -1],
    [-1, 5, -1,    -1, -1, -1,     7, -1, -1],

    [-1, 3, -1,     -1, -1, 1,    -1, 2, -1],
    [-1, -1, -1,    -1, -1, -1,    -1, -1, -1],
    [-1, -1, -1,     -1, -1, -1,   -1, -1, 1]
]
ext.populate(puzzle)
solve(ext, True)

Grid is good!
[2, 4, 3, 1, 6, 7, 5, 8, 9]
[1, 6, 7, 5, 8, 9, 3, 4, 2]
[5, 8, 9, 2, 3, 4, 6, 1, 7]
[3, 2, 4, 6, 7, 5, 1, 9, 8]
[6, 7, 1, 9, 4, 8, 2, 3, 5]
[9, 5, 8, 3, 1, 2, 7, 6, 4]
[4, 3, 5, 7, 9, 1, 8, 2, 6]
[8, 1, 2, 4, 5, 6, 9, 7, 3]
[7, 9, 6, 8, 2, 3, 4, 5, 1]
243167589
167589342
589234617
324675198
671948235
958312764
435791826
812456973
796823451
9
GCD:  9
Grid is good!
[2, 4, 3, 1, 6, 7, 5, 8, 9]
[1, 6, 7, 5, 8, 9, 3, 4, 2]
[5, 8, 9, 2, 3, 4, 6, 1, 7]
[3, 2, 4, 6, 7, 5, 1, 9, 8]
[9, 7, 1, 3, 4, 8, 2, 6, 5]
[6, 5, 8, 9, 1, 2, 7, 3, 4]
[4, 3, 5, 7, 9, 1, 8, 2, 6]
[8, 1, 2, 4, 5, 6, 9, 7, 3]
[7, 9, 6, 8, 2, 3, 4, 5, 1]
243167589
167589342
589234617
324675198
971348265
658912734
435791826
812456973
796823451
9
GCD:  9
Grid is good!
[2, 4, 3, 1, 6, 7, 5, 8, 9]
[1, 6, 7, 5, 8, 9, 3, 4, 2]
[5, 8, 9, 2, 3, 4, 6, 1, 7]
[3, 2, 4, 6, 7, 8, 1, 9, 5]
[6, 7, 1, 9, 4, 5, 2, 3, 8]
[9, 5, 8, 3, 1, 2, 7, 6, 4]
[4, 3, 5, 7, 9, 1, 8, 2, 6]
[8, 1, 2, 4, 5, 6, 9, 7, 3]
[7, 9, 6, 8, 2, 3, 4, 5, 1]
24

## Jane Street puzzle grid version

In [141]:
class JaneStreetGrid:
    def __init__(self):
        self.raw_grid = [[Cell() for c in range(9)] for r in range (9)]
        self.grid_t = []
        self.legal_values = []
        
        # Stack of cells for which a possible value was attempted.
        # Will contain a list of coordinate [r, c] of the last cell that we attempted to resolve
        self.prev_cell = []

    def populate(self, new_grid):
        self.raw_grid = [ [Cell(new_grid[r][c]) for c in range (9)] for r in range (9) ]

    def transpose(self):
        if (len(self.grid_t) == 0):
            self.grid_t = [ [self.raw_grid[r][c] for r in range(9)] for c in range (9) ]
    
    def is_set_valid(self, cells):
        values = [i.get_val() for i in cells]
        # all_values = [i for i in range(1, 10)]
        valid = True if len(set(values).symmetric_difference(self.legal_values)) == 0 else False
        # if valid == False:
        #    print ('{} contains non-unique elements'.format(values))
        return (valid)

    def check_grid(self):
        # Check all rows
        r_idx = 0
        for r in self.raw_grid:
            valid = self.is_set_valid(r)
            if (valid == False):
                # print ('Row Index: {} = {} does not contain all digits from 0-9'.format(r_idx, r))
                return (False)
            r_idx += 1

        # Check all columns
        self.transpose()
        c_idx = 0
        for c in self.grid_t:
            valid = self.is_set_valid(c)
            if (valid == False):
                # print ('Column Index: {} = {} does not contain all digits from 0-9'.format(c, c_idx))
                return (False)
            c_idx += 1


        # Check all sub-grids
        row_idx = 0
        col_idx = 0
        for grid_c in range (3):
            for grid_r in range (3):
                subgrid = [[self.raw_grid[r][c] for c in range (col_idx, col_idx+3)] for r in range (row_idx, row_idx+3)]
                # print (subgrid)
                subgrid_flat = [i for r in subgrid for i in r]
                valid = self.is_set_valid(subgrid_flat)
                if (valid == False):
                    # print ('Subgrid {} x {} = {} does not contain all digits from 0-9'.format(grid_r, grid_c, subgrid))
                    return (False)
                col_idx += 3

            col_idx = 0
            row_idx += 3

        # print ('\nGrid is good!')
        return (True)
        
        
    def print(self):
        for r in self.raw_grid:
            print (r)

    def print_transpose(self):
        if len(self.grid_t) == 0: 
            self.transpose()

        for r in self.grid_t:
            print (r) 

    # Solver functions
    # For a given cell, we need to know what's in the rows, columns corresponding to the subgrid the cell is in, and the subgrid itself.
    # Decision: 
    # - Loop through all empty cells
    # - For an empty cell, 
    def old_solve(self):
        while (self.check_grid() == False):
            for r in range (9):
                for c in range (9):
                    v = self.raw_grid[r][c] # Pass by reference
                    if v.get_val() != -1:
                        continue
                    row = self.raw_grid[r]

                    self.transpose()
                    col = self.grid_t[c]

                    row_idx = int(r / 3) * 3 
                    col_idx = int(c / 3) * 3; 
                    subgrid = [[self.raw_grid[r][c] for c in range (col_idx, col_idx+3)] for r in range (row_idx, row_idx+3)]
                    subgrid_flat = [i for r in subgrid for i in r]

                    # print ("Value = {}, Row = {}, Column = {}, subgrid = {}".format(v, row, col, subgrid))

                    # TODO FIXME all_poss = [i for i in range (10)] # For this problem it is in 0-9
                    all_poss = [i for i in range (1, 10)] # For an actual sudoku, this would be 1-9
                    # print (all_poss)
                    cannot_be_values = row + col + subgrid_flat
                    unique_cannot_be_values = []
                    for ii in cannot_be_values:
                        if ii.get_val() not in unique_cannot_be_values:
                            unique_cannot_be_values.append(ii.get_val())

                    v.possible_values = list(set(all_poss).symmetric_difference(unique_cannot_be_values))
                    if (-1 in v.possible_values):
                        v.possible_values.remove(-1)
                    
                    # print (v.possible_values)

                    if (len(v.possible_values) == 1):
                        v.value = v.possible_values.pop()



    def generate_possible_values(self, r, c):
        row = self.raw_grid[r]

        self.transpose()
        col = self.grid_t[c]

        row_idx = int(r / 3) * 3 
        col_idx = int(c / 3) * 3; 
        subgrid = [[self.raw_grid[r][c] for c in range (col_idx, col_idx+3)] for r in range (row_idx, row_idx+3)]
        subgrid_flat = [i for r in subgrid for i in r]

        # print ("Value = {}, Row = {}, Column = {}, subgrid = {}".format(v, row, col, subgrid))

        # Will get all possible values from self.legal_values
        # all_poss = [i for i in range (1, 10)] # For an actual sudoku, this is 1-9
        # print (all_poss)
        cannot_be_values = row + col + subgrid_flat
        unique_cannot_be_values = []
        for ii in cannot_be_values:
            if ii.get_val() not in unique_cannot_be_values:
                unique_cannot_be_values.append(ii.get_val())

        self.raw_grid[r][c].possible_values = list(set(self.legal_values).symmetric_difference(unique_cannot_be_values))
        if (-1 in self.raw_grid[r][c].possible_values):
            self.raw_grid[r][c].possible_values.remove(-1)
        
        return (self.raw_grid[r][c].possible_values)






In [156]:
puzzle = [
    [-1, -1, -1,     -1, -1, -1,     -1, 2, -1],
    [-1, -1, -1,    -1, -1, -1,     -1, -1, 5],
    [-1, 2, -1,     -1, -1, -1,    -1, -1, -1],

    [-1, -1, 0,    -1, -1, -1,    -1, -1, -1],
    [-1, -1, -1,     -1, -1, -1,     -1, -1, -1],
    [-1, -1, -1,    2, -1, -1,     -1, -1, -1],

    [-1, -1, -1,     -1, 0, 1,    -1, -1, -1],
    [-1, -1, -1,    -1, -1, 2,    -1, -1, -1],
    [-1, -1, -1,     -1, -1, -1,   5, -1, 1]
]

trials = [1, 3, 4, 6, 7, 8, 9] # 0, 2, 5 must be in the grid
full_set = [i for i in range (10)] # full range of 0-9

# for i in trials:
full_set.remove(1)

ext = JaneStreetGrid()
ext.legal_values = [0, 1, 2, 3, 4, 5, 6, 7, 8] # full_set

print (f"Solving with the following: {ext.legal_values}")

ext.populate(puzzle)
solve(ext, True)

full_set.append(1) # Making the set whole again

Solving with the following: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Grid is good!
[0, 1, 3, 4, 5, 6, 7, 2, 8]
[4, 6, 7, 0, 2, 8, 1, 3, 5]
[5, 2, 8, 1, 3, 7, 0, 4, 6]
[1, 3, 0, 6, 4, 5, 2, 8, 7]
[2, 4, 5, 7, 8, 0, 6, 1, 3]
[7, 8, 6, 2, 1, 3, 4, 5, 0]
[3, 7, 4, 5, 0, 1, 8, 6, 2]
[6, 5, 1, 8, 7, 2, 3, 0, 4]
[8, 0, 2, 3, 6, 4, 5, 7, 1]
13456728
467028135
528137046
130645287
245780613
786213450
374501862
651872304
802364571
9
GCD:  9
Grid is good!
[0, 1, 3, 4, 5, 6, 7, 2, 8]
[4, 6, 7, 0, 2, 8, 1, 3, 5]
[5, 2, 8, 1, 3, 7, 0, 4, 6]
[1, 3, 0, 6, 4, 5, 2, 8, 7]
[2, 4, 5, 7, 8, 3, 6, 1, 0]
[7, 8, 6, 2, 1, 0, 4, 5, 3]
[3, 7, 4, 5, 0, 1, 8, 6, 2]
[6, 5, 1, 8, 7, 2, 3, 0, 4]
[8, 0, 2, 3, 6, 4, 5, 7, 1]
13456728
467028135
528137046
130645287
245783610
786210453
374501862
651872304
802364571
9
GCD:  9
Grid is good!
[0, 1, 3, 4, 5, 6, 7, 2, 8]
[4, 6, 7, 0, 2, 8, 1, 3, 5]
[5, 2, 8, 1, 3, 7, 0, 4, 6]
[1, 3, 0, 6, 4, 5, 2, 8, 7]
[2, 8, 6, 7, 1, 0, 4, 5, 3]
[7, 4, 5, 2, 8, 3, 6, 1, 0]
[3, 7, 4, 5, 0, 1, 8, 6, 2]
[6, 